# __Predicting Outcomes of Call Option Contracts__

# SECTION 1: Preparation

##  1.1.) Loading the python packages

In [1]:

from sklearn import datasets

# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
import random

# Load libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV
from evolutionary_search import EvolutionaryAlgorithmSearchCV

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier

#importing classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import confusion_matrix
#from imblearn.metrics import classification_report_imbalanced

#Libraries for Saving the Model
from pickle import dump
from pickle import load

import warnings
warnings.filterwarnings('ignore')



## 1.2.) Loading the Data

In [2]:
#NOTE: This iris dataset is for debugging purposes only

#iris = datasets.load_iris()

#X = iris.data

#y = iris.target

#validation_size = 0.2

#seed = 7

In [3]:
#Import call options info

df = pd.read_csv(
    Path("../Resources/train.csv")
)


In [4]:
df.head()

,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.62598,0.05055,0.21136,-0.28773,0.03788,0.13515,...,-2.43,0.8,0.55,-87.089202,0.0,17.68,0.196649,0.0258,0,0
1,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.01171,0.00474,0.01717,-0.01484,0.00056,0.11400,...,14.57,0.8,0.01,0.000000,2.0,17.68,0.196649,0.0258,0,0
2,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.01569,0.00635,0.02257,-0.01973,0.00132,0.11196,...,13.57,0.8,0.02,0.000000,2.0,17.68,0.196649,0.0258,0,0
3,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.02331,0.00888,0.03151,-0.02878,0.00105,0.11290,...,12.57,0.8,0.01,-66.666667,0.0,17.68,0.196649,0.0258,0,0
4,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.02852,0.01072,0.03713,-0.03303,0.00202,0.10768,...,11.57,0.8,0.01,-75.000000,0.0,17.68,0.196649,0.0258,0,0


# SECTION 2: Exploratory Data Analysis

## 2.1.) Descriptive Statistics

In [5]:
df.shape

(11600, 24)

In [6]:
#Check for any null values and remove the null values
print('Null Values =', df.isnull().values.any())

#Drop NaNs
df.dropna()

Null Values = True


,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.62598,0.05055,0.21136,-0.28773,0.03788,0.13515,...,-2.43,0.8,0.55,-87.089202,0.0,17.68,0.196649,0.0258,0,0
1,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.01171,0.00474,0.01717,-0.01484,0.00056,0.11400,...,14.57,0.8,0.01,0.000000,2.0,17.68,0.196649,0.0258,0,0
2,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.01569,0.00635,0.02257,-0.01973,0.00132,0.11196,...,13.57,0.8,0.02,0.000000,2.0,17.68,0.196649,0.0258,0,0
3,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.02331,0.00888,0.03151,-0.02878,0.00105,0.11290,...,12.57,0.8,0.01,-66.666667,0.0,17.68,0.196649,0.0258,0,0
4,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.02852,0.01072,0.03713,-0.03303,0.00202,0.10768,...,11.57,0.8,0.01,-75.000000,0.0,17.68,0.196649,0.0258,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11595,2022-03-30 16:00:00,458.41,2022-04-01,2.00,0.85339,0.03203,0.09145,-0.29690,0.02853,0.18133,...,-7.41,0.8,7.75,0.000000,2.0,19.59,0.159647,0.0516,0,0
11596,2022-03-30 16:00:00,458.41,2022-04-01,2.00,0.88476,0.02729,0.07670,-0.23919,0.03003,0.18024,...,-8.41,0.8,7.80,0.000000,2.0,19.59,0.159647,0.0516,0,0
11597,2022-03-30 16:00:00,458.41,2022-04-01,2.00,0.08595,0.02864,0.06192,-0.13948,0.00262,0.13803,...,7.59,0.8,0.21,0.000000,2.0,19.59,0.159647,0.0516,0,0
11598,2022-03-30 16:00:00,458.41,2022-04-01,2.00,0.59355,0.06276,0.15405,-0.48771,0.01968,0.15576,...,-1.41,0.8,3.25,0.000000,2.0,19.59,0.159647,0.0516,0,0


In [7]:
display(df.head())
df.tail()

,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.62598,0.05055,0.21136,-0.28773,0.03788,0.13515,...,-2.43,0.8,0.55,-87.089202,0.0,17.68,0.196649,0.0258,0,0
1,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.01171,0.00474,0.01717,-0.01484,0.00056,0.11400,...,14.57,0.8,0.01,0.000000,2.0,17.68,0.196649,0.0258,0,0
2,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.01569,0.00635,0.02257,-0.01973,0.00132,0.11196,...,13.57,0.8,0.02,0.000000,2.0,17.68,0.196649,0.0258,0,0
3,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.02331,0.00888,0.03151,-0.02878,0.00105,0.11290,...,12.57,0.8,0.01,-66.666667,0.0,17.68,0.196649,0.0258,0,0
4,2022-01-03 09:30:00,476.43,2022-01-07,4.27,0.02852,0.01072,0.03713,-0.03303,0.00202,0.10768,...,11.57,0.8,0.01,-75.000000,0.0,17.68,0.196649,0.0258,0,0


,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
11595,2022-03-30 16:00:00,458.41,2022-04-01,2.0,0.85339,0.03203,0.09145,-0.29690,0.02853,0.18133,...,-7.41,0.8,7.75,0.0,2.0,19.59,0.159647,0.0516,0,0
11596,2022-03-30 16:00:00,458.41,2022-04-01,2.0,0.88476,0.02729,0.07670,-0.23919,0.03003,0.18024,...,-8.41,0.8,7.80,0.0,2.0,19.59,0.159647,0.0516,0,0
11597,2022-03-30 16:00:00,458.41,2022-04-01,2.0,0.08595,0.02864,0.06192,-0.13948,0.00262,0.13803,...,7.59,0.8,0.21,0.0,2.0,19.59,0.159647,0.0516,0,0
11598,2022-03-30 16:00:00,458.41,2022-04-01,2.0,0.59355,0.06276,0.15405,-0.48771,0.01968,0.15576,...,-1.41,0.8,3.25,0.0,2.0,19.59,0.159647,0.0516,0,0
11599,2022-03-30 16:00:00,458.41,2022-04-01,2.0,0.06393,0.02254,0.05007,-0.10798,0.00251,0.14098,...,8.59,0.8,0.14,0.0,2.0,19.59,0.159647,0.0516,0,0


In [8]:
df.describe()

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,...,STRIKE DISTANCE,INFLATION%,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
count,11600.000000,11600.000000,11600.000000,11600.000000,11600.000000,11600.000000,11600.000000,11596.000000,11598.000000,11600.000000,...,11600.000000,11600.000000,7945.000000,7945.000000,7945.000000,11600.000000,11600.000000,11600.000000,11600.000000,11600.000000
mean,449.944948,3.307968,0.333623,0.031125,0.128791,-0.330406,0.014387,0.208106,4666.413951,2.858147,...,4.561517,0.620914,3.356904,7.916701,1.857143,23.869617,0.059941,0.010448,0.009655,0.009655
std,13.736123,0.792421,0.243997,0.015543,0.057625,0.212916,0.010182,0.081595,6761.375543,3.083057,...,7.465352,0.131838,4.151342,112.933417,1.772313,4.410323,0.120807,0.092575,0.277768,0.277768
min,410.870000,2.000000,0.001990,0.000000,0.000000,-1.063160,-0.000350,0.080550,0.000000,0.000000,...,-18.400000,0.500000,0.010000,-97.701149,0.000000,16.520000,-0.218369,-0.318200,-1.000000,-1.000000
25%,440.640000,3.000000,0.115058,0.020397,0.086527,-0.459310,0.005570,0.155128,662.250000,0.430000,...,-0.720000,0.500000,0.170000,-72.431078,0.000000,20.500000,0.007646,0.000000,0.000000,0.000000
50%,449.420000,3.210000,0.299620,0.030365,0.142775,-0.316050,0.013070,0.185175,2277.000000,1.750000,...,4.600000,0.600000,1.430000,-19.230769,1.000000,23.100000,0.089130,0.000000,0.000000,0.000000
75%,458.990000,4.100000,0.519498,0.041210,0.176470,-0.165288,0.022050,0.242317,5806.250000,4.380000,...,9.850000,0.800000,5.350000,37.759815,3.000000,26.990000,0.147081,0.000000,0.000000,0.000000
max,479.650000,4.270000,1.000000,0.098060,0.221910,0.000000,0.047520,0.495160,67266.000000,19.370000,...,29.840000,0.800000,18.470000,912.500000,5.000000,37.980000,0.307204,0.340000,1.000000,1.000000


In [9]:
df.dtypes

QUOTE_READTIME          object
SPY PRICE              float64
EXPIRE_DATE             object
DTE                    float64
C_DELTA                float64
C_GAMMA                float64
C_VEGA                 float64
C_THETA                float64
C_RHO                  float64
C_IV                   float64
C_VOLUME               float64
C_LAST                 float64
STRIKE                 float64
STRIKE_DISTANCE_PCT    float64
STRIKE DISTANCE        float64
INFLATION%             float64
PRICECLOSE             float64
ROI %                  float64
y                      float64
VIX PRICE              float64
AVG SCORE              float64
MED SCORE              float64
SEN IND AVG NUM          int64
SEN IND MED NUM          int64
dtype: object

## 2.2.) Feature Analysis and Exploration

### Plotting features according to contract length

In [10]:
#
#contract_outcome = df.groupby('STRIKE_DISTANCE_PCT')['y'].value_counts(normalize=True).loc[:,1]
#sns.set(rc={'figure.figsize':(12,5)})
#sns.barplot(x=contract_outcome.index, y=contract_outcome.values, color='#5975A4', saturation=1)

### Eliminate Uncorrelated Features

In [11]:
#Calculate correlation of each feature with 'y'

correlation = df.corr()
correlation_df = abs(correlation['y'])

In [12]:
correlation_df.sort_values(ascending=False)

y                      1.000000
ROI %                  0.863641
PRICECLOSE             0.526526
INFLATION%             0.348653
SEN IND AVG NUM        0.278600
DTE                    0.176449
C_VEGA                 0.148314
C_RHO                  0.145546
SEN IND MED NUM        0.122381
STRIKE DISTANCE        0.121717
C_DELTA                0.119877
C_LAST                 0.109332
MED SCORE              0.100527
C_VOLUME               0.095233
STRIKE_DISTANCE_PCT    0.084792
STRIKE                 0.082254
SPY PRICE              0.022821
VIX PRICE              0.021021
C_THETA                0.010303
AVG SCORE              0.008157
C_IV                   0.005216
C_GAMMA                0.005073
Name: y, dtype: float64

In [13]:
#Drop variables with less than 3% correlation with contract outcome ('y')

drop_list_corr = sorted(list(correlation_df[correlation_df < 0.03].index))
print(drop_list_corr)

['AVG SCORE', 'C_GAMMA', 'C_IV', 'C_THETA', 'SPY PRICE', 'VIX PRICE']


In [14]:
#Dropping because C_IV convolutes outcome for some buggish reason

df.drop(labels=["C_IV"], axis=1, inplace=True)

### Drop Columns Not Needed for Machine Learning

In [15]:
df.drop(columns=["QUOTE_READTIME"], inplace=True)

In [16]:
df.drop(columns=["ROI %"], inplace=True)

In [17]:
df.drop(columns=["PRICECLOSE"], inplace=True)

In [18]:
df.drop(columns=["EXPIRE_DATE"], inplace=True)

In [19]:
df.drop(columns=["C_VOLUME"], inplace=True)

# SECTION 3: Evaluate Algorithms and Models

## 3.1.) Train Test Split and Evaluation Metrics

In [20]:
#Import call options info for March as testing data

test = pd.read_csv(
    Path("../Resources/test_ready.csv")
)


In [21]:
test.drop(columns=["Unnamed: 0"], inplace=True)

In [22]:
test.drop(columns=["C_VOLUME"], inplace=True)

In [23]:
test

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,INFLATION%,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,420.68,4.06,0.67743,0.02409,0.16594,-0.45141,0.02360,10.72,412.0,0.021,-8.68,0.6,NaN,32.01,-0.044775,0.0,0,0
1,420.68,4.06,0.65631,0.02476,0.17042,-0.46023,0.02290,8.76,413.0,0.018,-7.68,0.6,NaN,32.01,-0.044775,0.0,0,0
2,420.68,4.06,0.63414,0.02544,0.17446,-0.46849,0.02288,8.80,414.0,0.016,-6.68,0.6,NaN,32.01,-0.044775,0.0,0,0
3,420.68,4.06,0.69735,0.02335,0.16063,-0.44086,0.02317,10.93,411.0,0.023,-9.68,0.6,NaN,32.01,-0.044775,0.0,0,0
4,423.38,4.04,0.63464,0.02711,0.17433,-0.43781,0.02282,8.77,417.0,0.015,-6.38,0.6,NaN,30.86,-0.044775,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,433.31,2.21,0.84058,0.02339,0.08571,-0.34777,0.01188,14.39,419.0,0.033,-14.31,0.8,2.0,27.58,0.089130,0.0,0,0
2896,433.31,2.21,0.83081,0.02509,0.08944,-0.35051,0.01248,13.48,420.0,0.031,-13.31,0.8,3.0,27.58,0.089130,0.0,0,0
2897,433.31,2.21,0.69645,0.03227,0.12836,-0.52916,0.01394,8.71,426.0,0.017,-7.31,0.8,3.0,27.58,0.089130,0.0,0,0
2898,433.31,2.21,0.78689,0.02699,0.10522,-0.43651,0.01315,11.82,422.0,0.026,-11.31,0.8,3.0,27.58,0.089130,0.0,0,0


In [24]:
#Split into training and testing groups

#Split out validation dataset for the end
y_train = df["y"]

X_train = df.drop(columns='y')


In [25]:
y_train.value_counts()

0.0    2451
1.0    1788
5.0    1171
3.0    1100
2.0     964
4.0     471
Name: y, dtype: int64

In [26]:
#Split into training and testing groups

#Split out validation dataset for the end
y_validation = test["y"]

X_validation = test.drop(columns='y')


In [27]:

#validation_size = 0.2

#seed = 7

#X_train, y_train = train_test_split(X, y, stratify=y, random_state=seed)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [28]:
X_validation

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,INFLATION%,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,420.68,4.06,0.67743,0.02409,0.16594,-0.45141,0.02360,10.72,412.0,0.021,-8.68,0.6,32.01,-0.044775,0.0,0,0
1,420.68,4.06,0.65631,0.02476,0.17042,-0.46023,0.02290,8.76,413.0,0.018,-7.68,0.6,32.01,-0.044775,0.0,0,0
2,420.68,4.06,0.63414,0.02544,0.17446,-0.46849,0.02288,8.80,414.0,0.016,-6.68,0.6,32.01,-0.044775,0.0,0,0
3,420.68,4.06,0.69735,0.02335,0.16063,-0.44086,0.02317,10.93,411.0,0.023,-9.68,0.6,32.01,-0.044775,0.0,0,0
4,423.38,4.04,0.63464,0.02711,0.17433,-0.43781,0.02282,8.77,417.0,0.015,-6.38,0.6,30.86,-0.044775,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,433.31,2.21,0.84058,0.02339,0.08571,-0.34777,0.01188,14.39,419.0,0.033,-14.31,0.8,27.58,0.089130,0.0,0,0
2896,433.31,2.21,0.83081,0.02509,0.08944,-0.35051,0.01248,13.48,420.0,0.031,-13.31,0.8,27.58,0.089130,0.0,0,0
2897,433.31,2.21,0.69645,0.03227,0.12836,-0.52916,0.01394,8.71,426.0,0.017,-7.31,0.8,27.58,0.089130,0.0,0,0
2898,433.31,2.21,0.78689,0.02699,0.10522,-0.43651,0.01315,11.82,422.0,0.026,-11.31,0.8,27.58,0.089130,0.0,0,0


In [29]:
X_train

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,INFLATION%,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,476.43,4.27,0.62598,0.05055,0.21136,-0.28773,0.03788,4.26,474.0,0.005,-2.43,0.8,17.68,0.196649,0.0258,0,0
1,476.43,4.27,0.01171,0.00474,0.01717,-0.01484,0.00056,0.01,491.0,0.031,14.57,0.8,17.68,0.196649,0.0258,0,0
2,476.43,4.27,0.01569,0.00635,0.02257,-0.01973,0.00132,0.02,490.0,0.028,13.57,0.8,17.68,0.196649,0.0258,0,0
3,476.43,4.27,0.02331,0.00888,0.03151,-0.02878,0.00105,0.03,489.0,0.026,12.57,0.8,17.68,0.196649,0.0258,0,0
4,476.43,4.27,0.02852,0.01072,0.03713,-0.03303,0.00202,0.04,488.0,0.024,11.57,0.8,17.68,0.196649,0.0258,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11595,458.41,2.00,0.85339,0.03203,0.09145,-0.29690,0.02853,7.75,451.0,0.016,-7.41,0.8,19.59,0.159647,0.0516,0,0
11596,458.41,2.00,0.88476,0.02729,0.07670,-0.23919,0.03003,7.80,450.0,0.018,-8.41,0.8,19.59,0.159647,0.0516,0,0
11597,458.41,2.00,0.08595,0.02864,0.06192,-0.13948,0.00262,0.21,466.0,0.017,7.59,0.8,19.59,0.159647,0.0516,0,0
11598,458.41,2.00,0.59355,0.06276,0.15405,-0.48771,0.01968,3.25,457.0,0.003,-1.41,0.8,19.59,0.159647,0.0516,0,0


In [30]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_validation_scaled = X_scaler.transform(X_validation)

## 3.2.) Quick Check of Models and Algorithms

In [31]:
#Spot check Classification algorithms

models = []

#Boosting methods

models.append(('XGB', XGBClassifier()))
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))

#Bagging methods

models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))

#SVC 
models.append(('SVC', SVC()))

In [32]:
#Test options for classification

num_folds = 10
seed = 7

In [33]:
results = []

names = []

scoring = 'accuracy'

for name, model in models:
    kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    findings_summary = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(findings_summary)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#Compare the model outcomes

fig = pyplot.figure()
fig.suptitle('Model Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
fig.set_size_inches(8,4)
pyplot.show()

## 3.3.) Selecting Random Forest (or whichver tree method best), Calculating its Baseline 

In [ ]:
#Estimate accuracy on training set

rf_model = RandomForestClassifier()

cv = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
scores = cross_val_score(rf_model, X_train_scaled, y_train, scoring='accuracy', cv=cv)
avg_score = np.mean(scores)
avg_score

In [ ]:
#Estimate accuracy on validation set

rf_model = RandomForestClassifier()

rf_model.fit(X_train_scaled, y_train)

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

# SECTION 4: Model Tuning 

## 4.1.) Hyperparameter Tuning for Random Forest

### APPROACH (A) 
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {}

paramgrid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,}
              # 'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
              # 'bootstrap': bootstrap}

random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=RandomForestClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=4),
                                   verbose=1,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=4)
cv.fit(X_train_scaled, y_train)

### APPROACH (B)
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:
# Grid Search: (select model) Tuning

n_estimators = [20, 100, 180, 1000]
max_features = [1, 2, 3, 4]

param_grid = dict(n_estimators=n_estimators, max_features=max_features)

rf_model = RandomForestClassifier()

kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)

grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X_train_scaled, y_train)

#Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
ranks = grid_result.cv_results_['rank_test_score']
for mean, stdev, param, rank in zip(means, stds, params, ranks):
    print("#%d %f (%f) with: %r" % (rank, mean, stdev, param))

### PROCEEDING WITH BEST APPROACH

In [ ]:
#Prepare the model with parameters decided in previous cell

rf_model = RandomForestClassifier() #(n_estimators=200, max_features="sqrt", max_depth=100, min_samples_split=5, min_samples_leaf=1, bootstrap=False)
rf_model.fit(X_train_scaled, y_train)

In [ ]:
#Score predictions of training set

training_predictions = rf_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))


print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Score predictions of validation set

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))


print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

## 4.2.) Determining Feature Importance

In [ ]:
#Feature importances

importances = rf_model.feature_importances_

In [ ]:
important_features = zip(X.columns, rf_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

## 4.2. Bayesian Optimization with HYPEROPT for XGB

## __*(CAUTION TAKES A COUPLE OF MIN!)*__

In [ ]:
#Initialize domain space for range of values 
 
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1, 9),
        'reg_alpha' : hp.quniform('reg_alpha', 40, 180, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }


The available hyperopt optimization algorithms are -

hp.choice(label, options) — Returns one of the options, which should be a list or tuple.

hp.randint(label, upper) — Returns a random integer between the range [0, upper).

hp.uniform(label, low, high) — Returns a value uniformly between low and high.

hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer.

hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

In [ ]:
def objective(space):
    xgb_model = xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [(X_train_scaled, y_train), (X_validation_scaled, y_validation)]
    
    xgb_model.fit(X_train_scaled, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = xgb_model.predict(X_validation_scaled)
    accuracy = accuracy_score(y_validation, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
#Establish XGB model instance
#Fitting training dataset to model

xgb_model = xgb.XGBClassifier() #colsample_bytree=0.8056144829851541, max_depth=8, gamma=6.979438878191186, min_child_weight=10.0, reg_lambda=0.00013734197899190276, reg_alpha=0)
xgb_model.fit(X_train_scaled, y_train)

In [ ]:
#Predicting on training set

training_predictions = xgb_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))

print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Predicting on validation set

predictions = xgb_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Get probabilities of predictions

pred_proba = xgb_model.predict_proba(X_validation_scaled)
pred_proba

In [ ]:
proba_df = pd.DataFrame(pred_proba.round(2))

In [ ]:
#Formatting probabilities DataFrame

proba_df["Actual"] = y_validation.reset_index(drop=True)
proba_df.columns = ["'Strong Sell' Probability", "'Sell' Probability", "'Pass' Probability", "'Buy' Probability", "'Strong Buy' Probability", "'Very High Return' Prob.", "Actual"]
proba_df

In [ ]:
#Labeling caterogires

proba_df.loc[proba_df["Actual"] == 0, "Actual"] = "Strong Sell"
proba_df.loc[proba_df["Actual"] == 1, "Actual"] = "Sell"
proba_df.loc[proba_df["Actual"] == 2, "Actual"] = "Pass"
proba_df.loc[proba_df["Actual"] == 3, "Actual"] = "Buy"
proba_df.loc[proba_df["Actual"] == 4, "Actual"] = "Strong Buy"
proba_df.loc[proba_df["Actual"] == 5, "Actual"] = "Very High Return"

proba_df.head(50)

In [ ]:
#Get ROC AUC score

roc_auc_score(y_validation, pred_proba, multi_class="ovr")

## 4.2.) Determining Feature Importance for XGBoost Model

In [ ]:
#Feature importances

importances = xgb_model.feature_importances_

In [ ]:
important_features = zip(X.columns, xgb_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

In [ ]:
#et_model = ExtraTreesClassifier()
#et_model.fit(X_train_scaled, y_train)

In [ ]:
#ab_model = AdaBoostClassifier()
#ab_model.fit(X_train_scaled, y_train)

In [ ]:
#gb_model = GradientBoostingClassifier()
#ab_model.fit(X_train_scaled, y_train)